In [2]:
%matplotlib notebook
import numpy as np
MEPS = np.finfo(float).eps
import scipy.linalg as la
import mpl_toolkits.mplot3d
import matplotlib.pyplot as plt

from scipy.spatial import cKDTree

import scipy.sparse as sp
from scipy.sparse.linalg import spsolve, lsqr

from numpy.linalg import cond, norm, inv


import pandas as pd

import pickle
import os.path

import warnings
warnings.simplefilter('ignore')

from scipy.optimize import minimize_scalar

from poly_basis import *
from spherepts import *
from rbf import *

In [142]:
def dist_outer(nodes1, nodes2, W):
    ret = [ [(n1-n2) @ W @ (n1-n2) for n1 in nodes1] for n2 in nodes2]
    return np.sqrt(ret)

In [171]:
# Choose RBF
rbf_obj = rbf_dict['gaussian']
zeta  = rbf_obj['zeta']
d2rbf = rbf_obj['d2rbf']
Lrbf = lambda r,eps: 2*zeta(r,eps) + d2rbf(r,eps)

# Choose solution and forcing fuction
solution_index = 20
sol_deg = sphere_harm_degs[solution_index]
foo = lambda x: sphere_harm[solution_index](*x)*sol_deg*(sol_deg+1)
exact = lambda x: sphere_harm[solution_index](*x)
print('Harmonic degree: %d' % sphere_harm_degs[solution_index])

Harmonic degree: 4


In [172]:
n = 200
eps = 2
k = 20

nodes = gen_spiral_nodes(n)
Ws = np.zeros((n, 3, 3))
R = np.zeros((3, 3))
e1, e2, e3 = np.eye(3)
E = np.eye(3)
E[2,2] = 0
for i, node in enumerate(nodes):
    t1 = e2 - np.dot(node, e2)*node
    t1 /= la.norm(t1)
    t2 = e3 - np.dot(node, e3)*node - np.dot(t1, e3)*t1
    t2 /= la.norm(t2)
    R[0] = t1
    R[1] = t2
    R[2] = node
    Ws[i] = R.T @ E @ R

# Tangent Plane Method

In [173]:
my_node = n//3

W = Ws[my_node]

tree = cKDTree(np.array(nodes))
stencil = tree.query(nodes[my_node], k)[1]
nn = np.array([nodes[i] for i in stencil]) #nodes in the stencil

A = rbf(dist_outer(nn, nn, W), eps)
rhs = Lrbf(dist_outer(nn[0].reshape((1,3)), nn, W), eps)
ws = la.solve(A, rhs.flatten())

L_approx = np.dot(ws, [foo(n) for n in nn])
L_actual = exact(nn[0])

In [174]:
print(L_approx)
print(L_actual)
print((L_approx - L_actual)/L_actual)

-28.369006644203488
0.07405577187500167
-384.07624005441977


In [162]:
ws

array([[-93.06024999],
       [ 26.0802739 ],
       [ 25.94250877],
       [ 25.11678854],
       [ 22.68455274],
       [  4.4265188 ],
       [  3.22264584],
       [  1.37876794],
       [ -2.12475456],
       [ -2.62071888],
       [ -3.00391685],
       [ -3.03666699],
       [ -1.03341334],
       [ -1.88310619],
       [ -0.68724229],
       [ -1.36375183],
       [ -0.4636377 ],
       [ -0.33004129],
       [ -0.42796954],
       [  0.6626804 ]])